In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import time

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.


Setting environment for Precision WorkStation


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
! aws s3 cp --recursive s3://mousebrainatlas-rawdata/CSHL_data/MD589/ /data/CSHL_data/MD589/ --exclude "*" --include "*lossless.jp2"

# Rename "lossless" to "raw"

In [ ]:
for stack in all_nissl_stacks:

    for section in metadata_cache['valid_sections'][stack]:

        in_fp = \
        DataManager.get_image_filepath_v2(stack=stack, prep_id=2, section=section, version='jpeg', resol='lossless')

        out_fp = \
        DataManager.get_image_filepath_v2(stack=stack, prep_id=2, section=section, version='jpeg', resol='raw')

        create_parent_dir_if_not_exists(out_fp)
        
        execute_command('ln -s %s %s' % (in_fp, out_fp))

In [ ]:
for stack in all_nissl_stacks:

    for section in metadata_cache['valid_sections'][stack]:

        in_fp = \
        DataManager.get_image_filepath_v2(stack=stack, prep_id=2, section=section, version=None, resol='lossless')

        out_fp = \
        DataManager.get_image_filepath_v2(stack=stack, prep_id=2, section=section, version=None, resol='raw')

        create_parent_dir_if_not_exists(out_fp)
        
        execute_command('ln -s %s %s' % (in_fp, out_fp))

In [ ]:
for stack in all_nissl_stacks:

    for section in metadata_cache['valid_sections'][stack]:

        in_fp = \
        DataManager.get_image_filepath_v2(stack=stack, prep_id=2, section=section, version='gray', resol='lossless')

        out_fp = \
        DataManager.get_image_filepath_v2(stack=stack, prep_id=2, section=section, version='gray', resol='raw')

        create_parent_dir_if_not_exists(out_fp)
        
        execute_command('ln -s %s %s' % (in_fp, out_fp))

In [ ]:
stack = 'MD585'

In [ ]:
from skimage.transform import warp

In [ ]:
_, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack, redownload=True) 
valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]
print len(valid_filenames), 'valid filenames'

In [ ]:
def convert_image_types(from_type, to_type):
    
    from_prep_id = from_type['prep_id']
    to_prep_id = to_type['prep_id']
    
    if from_prep_id is None and to_prep_id == 1: # in-plane alignment
        pass
    
    elif (from_prep_id == 1 or from_prep_id == 'alignedPadded') and (to_prep_id == 5 or to_prep_id == 'alignedWithMargin'): # alignedPadded to alignedWithMargin
    
        alignedWithMargin_xmin, alignedWithMargin_xmax,\
        alignedWithMargin_ymin, alignedWithMargin_ymax = DataManager.load_cropbox_v2(stack=stack, anchor_fn=None, 
                                                            prep_id='alignedWithMargin',
                                                           return_dict=False, only_2d=True)
        
        in_fp = \
        DataManager.get_image_filepath_v2(stack=stack, prep_id=1, section=section, version=version, resol='thumbnail')

        out_fp = \
        DataManager.get_image_filepath_v2(stack=stack, prep_id=5, section=section, version=version, resol='thumbnail')

        create_parent_dir_if_not_exists(out_fp)

        t = time.time()
        im_prep1 = imread(in_fp)
        im_prep5 = im_prep1[alignedWithMargin_ymin:alignedWithMargin_ymax+1, 
                            alignedWithMargin_xmin:alignedWithMargin_xmax+1]        
        save_data(im_prep5, out_fp)            
        sys.stderr.write('Crop alignedPadded to generate alignedWithMargin: %.2f seconds.\n' % (time.time() - t))
        
    elif from_prep_id == 1 and to_prep_id == 2: # alignedWithMargin to alignedBrainstemCrop
        pass


In [ ]:
def convert_tif_to_jpeg(in_fp, out_fp):
    t = time.time()
    execute_command("convert %(in_fp)s %(out_fp)s" % {'in_fp': in_fp, 'out_fp': out_fp})
    sys.stderr.write('Convert to JPEG: %.2f seconds.\n' % (time.time() - t))

In [ ]:
# Convert to gray

for stack in ['MD585']:

    for section in metadata_cache['valid_sections'][stack]:

        in_fp = \
        DataManager.get_image_filepath_v2(stack=stack, prep_id=2, section=section, version=None, resol='raw')

        out_fp = \
        DataManager.get_image_filepath_v2(stack=stack, prep_id=2, section=section, version='gray', resol='raw')

        create_parent_dir_if_not_exists(out_fp)

#         t = time.time()
#         im_rgb = imread(in_fp)
#         sys.stderr.write('Load: %.2f seconds.\n' % (time.time() - t))
        
#         t = time.time()
#         im_gray = img_as_ubyte(rgb2gray(im_rgb))
# #         im_gray = im_rgb[..., 2]
#         sys.stderr.write('Convert RGB to gray: %.2f seconds.\n' % (time.time() - t))
        
#         t = time.time()
#         save_data(im_gray, out_fp)
#         sys.stderr.write('Save: %.2f seconds.\n' % (time.time() - t))

        t = time.time()
        im_rgb = imread(in_fp)
        sys.stderr.write('Load: %.2f seconds.\n' % (time.time() - t))
        
        t = time.time()
        im_gray = img_as_ubyte(rgb2gray(im_rgb))
#         im_gray = im_rgb[..., 2]
        sys.stderr.write('Convert RGB to gray: %.2f seconds.\n' % (time.time() - t))
        
        t = time.time()
        save_data(im_gray, out_fp, upload_s3=False)
        sys.stderr.write('Save: %.2f seconds.\n' % (time.time() - t))
        
#         upload_to_s3(out_fp)

In [ ]:
# Convert to JPEG

for stack in ['MD595']:
# for stack in all_nissl_stacks:

    for section in metadata_cache['valid_sections'][stack]:

        in_fp = \
        DataManager.get_image_filepath_v2(stack=stack, prep_id=2, section=section, version='gray', resol='raw')

        out_fp = \
        DataManager.get_image_filepath_v2(stack=stack, prep_id=2, section=section, version='grayJpeg', resol='raw')

        create_parent_dir_if_not_exists(out_fp)

        convert_tif_to_jpeg(in_fp, out_fp)
        
#         upload_to_s3(out_fp)

# Convert original alignedBrainstemCrop cropbox file to section limit json

In [ ]:
# for stack in ['MD658', 'MD661', 'MD662']:
for stack in ['MD635', 'MD653', 'MD652', 'MD642']:

    alignedBrainstemCrop_secmin, alignedBrainstemCrop_secmax = DataManager.load_cropbox(stack=stack)[4:]

    alignedBrainstemCrop_seclims = {'left_section_limit': alignedBrainstemCrop_secmin, 
                                    'right_section_limit': alignedBrainstemCrop_secmax}
    
    save_data(alignedBrainstemCrop_seclims, 
          DataManager.get_section_limits_filename_v2(stack=stack, anchor_fn=None, prep_id='alignedBrainstemCrop'))

# convert original alignedBrainstemCrop cropbox file to bbox json

In [25]:
# for stack in all_nissl_stacks:
# for stack in ['MD658', 'MD661', 'MD662']:
# for stack in ['MD635', 'MD653', 'MD652', 'MD642']:
for stack in ['MD657']:
    
    alignedBrainstemCrop_xmin, alignedBrainstemCrop_xmax, \
    alignedBrainstemCrop_ymin, alignedBrainstemCrop_ymax = DataManager.load_cropbox(stack=stack)[:4]

    alignedBrainstemCrop_cropbox = {'rostral_limit': alignedBrainstemCrop_xmin, 'caudal_limit': alignedBrainstemCrop_xmax, 
     'dorsal_limit': alignedBrainstemCrop_ymin, 'ventral_limit': alignedBrainstemCrop_ymax}
    
    save_data(alignedBrainstemCrop_cropbox, 
          DataManager.get_cropbox_filename_v2(stack=stack, anchor_fn=None, prep_id='alignedBrainstemCrop'))

aws s3 cp "/data/CSHL_data_processed/MD657/MD657_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_prep2_cropbox.json" "s3://mousebrainatlas-data/CSHL_data_processed/MD657/MD657_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_prep2_cropbox.json"
0


# Identify bounding box for "wholebrainWithMargin"

In [ ]:
tb_resol = 'down32'

In [ ]:
for stack in ['MD589']:
# for stack in all_nissl_stacks:
    
    bbox_all_images = []
    for fn in metadata_cache['valid_filenames'][stack]:
        mask_tb_alignedPadded = DataManager.load_thumbnail_mask_v3(stack=stack, prep_id=1, fn=fn)
        bbox = bbox_2d(mask_tb_alignedPadded)
        bbox_all_images.append(bbox)
    bbox_all_images = np.array(bbox_all_images)

#     bbox_all_images = np.array([
#         bbox_2d(DataManager.load_thumbnail_mask_v3(stack=stack, prep_id=1, fn=fn))
#         for fn in metadata_cache['valid_filenames'][stack]
#     ])
    
    # Are the bounding boxes reasonable? If some numbers stand out, go back to check the mask.
    plt.figure(figsize=(10,5));
    plt.plot(bbox_all_images[:,0], label='xmin')
    plt.plot(bbox_all_images[:,1], label='xmax')
    plt.plot(bbox_all_images[:,2], label='ymin')
    plt.plot(bbox_all_images[:,3], label='ymax')
    plt.legend();
    plt.show();
    
    margin_um = 736.
    margin = int(np.round(margin_um / convert_resolution_string_to_um(resolution=tb_resol, stack=stack)))
    alignedWithMargin_xmin, alignedWithMargin_ymin = np.maximum(bbox_all_images[:, [0,2]].min(axis=0) - margin, 0)
    alignedWithMargin_xmax, alignedWithMargin_ymax = np.minimum(bbox_all_images[:, [1,3]].max(axis=0) + margin, 
                                                                [mask_tb_alignedPadded.shape[1]-1, mask_tb_alignedPadded.shape[0]-1])
    
    print alignedWithMargin_xmin, alignedWithMargin_xmax, alignedWithMargin_ymin, alignedWithMargin_ymax 
    
    alignedWithMargin_cropbox = {'rostral_limit': alignedWithMargin_xmin, 'caudal_limit': alignedWithMargin_xmax, 
 'dorsal_limit': alignedWithMargin_ymin, 'ventral_limit': alignedWithMargin_ymax}
    
    save_data(alignedWithMargin_cropbox, 
              DataManager.get_cropbox_filename_v2(stack=stack, anchor_fn=None, prep_id='alignedWithMargin'))

*NOTE*: If any of the four sides looks out of place, identify the section and double-check its mask.

In [ ]:
def find_outlier(a, t, mode='max'):
    """
    Args:
        t(float): The peak must be different from neighbor by this much.
    """
    if mode == 'max':
        return np.r_[True, a[1:] - a[:-1] > t] & np.r_[a[:-1] - a[1:] > t, True]
    elif mode == 'min':
        return np.r_[True, a[1:] - a[:-1] < -t] & np.r_[a[:-1] - a[1:] < -t, True]
    else:
        raise

In [ ]:
# Identify sections whose bboxes are outliers.

print 'xmin outlier', [metadata_cache['valid_filenames'][stack][s] for s in np.where(find_outlier(bbox_all_images[:,0], 40, 'min'))[0]]
print 'xmax outlier', [metadata_cache['valid_filenames'][stack][s] for s in np.where(find_outlier(bbox_all_images[:,1], 40, 'max'))[0]]
print 'ymin outlier', [metadata_cache['valid_filenames'][stack][s] for s in np.where(find_outlier(bbox_all_images[:,2], 40, 'min'))[0]]
print 'ymax outlier', [metadata_cache['valid_filenames'][stack][s] for s in np.where(find_outlier(bbox_all_images[:,3], 40, 'max'))[0]]

In [ ]:
# for fn in metadata_cache['valid_filenames'][stack]:
#     mask_tb_alignedPadded = DataManager.load_thumbnail_mask_v3(stack=stack, prep_id=1, fn=fn)
#     plt.figure();
#     plt.imshow(mask_tb_alignedPadded[alignedWithMargin_ymin:alignedWithMargin_ymax+1, 
#                                      alignedWithMargin_xmin:alignedWithMargin_xmax+1], cmap=plt.cm.gray)
#     plt.show();

# Generate prep5 thumbnails from prep1 thumbnails

In [ ]:
for stack in all_nissl_stacks:

    alignedWithMargin_xmin, alignedWithMargin_xmax,\
    alignedWithMargin_ymin, alignedWithMargin_ymax = DataManager.load_cropbox_v2(stack=stack, anchor_fn=None, 
                                                            prep_id='alignedWithMargin',
                                                           return_dict=False, only_2d=True)
            
#     for section in metadata_cache['valid_sections_all'][stack]: # for CHATM2 and CHATM3
    for section in metadata_cache['valid_sections'][stack]: # for MD589 etc.
        
#         for version in ['NtbNormalized', 'mask']:
        for version in [None, 'mask']:

            in_fp = \
            DataManager.get_image_filepath_v2(stack=stack, prep_id=1, section=section, version=version, resol='thumbnail')

            out_fp = \
            DataManager.get_image_filepath_v2(stack=stack, prep_id=5, section=section, version=version, resol='thumbnail')

            create_parent_dir_if_not_exists(out_fp)

            t = time.time()

            im_prep1 = imread(in_fp)
            im_prep5 = im_prep1[alignedWithMargin_ymin:alignedWithMargin_ymax+1, 
                                alignedWithMargin_xmin:alignedWithMargin_xmax+1]        
            save_data(im_prep5, out_fp)
            
            sys.stderr.write('Generate prep5: %.2f seconds.\n' % (time.time() - t))

# Construct intensity volume

Also see `reconstruct/construct_intensity_volume_v3.ipynb`

In [ ]:
# output_resolution = 'down32'
output_resolution = '10.0um'

# tb_version = 'NtbNormalized'
tb_version = None
tb_resol = 'thumbnail'

In [ ]:
for stack in all_nissl_stacks:
# for stack in ['MD590']:

    images = {}

#     for sec in metadata_cache['valid_sections_all'][stack]:
    for sec in metadata_cache['valid_sections'][stack]:

        img_rgb = DataManager.load_image_v2(stack, section=sec, 
                                            resol=tb_resol, 
                                            prep_id='alignedWithMargin', 
                                            version=tb_version)
        img = img_as_ubyte(rgb2gray(img_rgb)) # Use greylevel

        mask = DataManager.load_image_v2(stack=stack, section=sec, 
                                         prep_id='alignedWithMargin', 
                                         resol=tb_resol, 
                                         version='mask')
        img[~mask] = 0

        images[sec] = img

    # Specify isotropic resolution of the output volume.
    voxel_size_um = convert_resolution_string_to_um(resolution=output_resolution, stack=stack)

    input_image_resolution_um = convert_resolution_string_to_um(resolution=tb_resol, stack=stack)

    volume_outVolResol, volume_origin_wrt_wholebrainWithMargin_outVolResol = images_to_volume_v2(images=images, 
                                                spacing_um=20.,
                                                in_resol_um=input_image_resolution_um,
                                                out_resol_um=voxel_size_um)
    print volume_outVolResol.shape
    
    ##############################################################
    
    prep5_origin_wrt_prep1_tbResol = DataManager.load_cropbox_v2(stack=stack, only_2d=True, prep_id='alignedWithMargin')

    loaded_cropbox_resol = 'thumbnail'

    prep5_origin_wrt_prep1_outVolResol = prep5_origin_wrt_prep1_tbResol * \
    convert_resolution_string_to_um(resolution=loaded_cropbox_resol, stack=stack) / voxel_size_um

    wholebrainWithMargin_origin_wrt_wholebrain_outVolResol = np.r_[np.round(prep5_origin_wrt_prep1_outVolResol).astype(np.int)[[0,2]], 0]
    # wholebrainWithMargin_origin_wrt_wholebrain = np.array([0,0,0])

    volume_origin_wrt_wholebrain_outVolResol = volume_origin_wrt_wholebrainWithMargin_outVolResol + wholebrainWithMargin_origin_wrt_wholebrain_outVolResol

    ########################################

    stack_spec = dict(name=stack,
                      resolution=output_resolution,
                      prep_id='wholebrainWithMargin',
                      vol_type='intensity')
    
    save_data(volume_outVolResol, 
              fp=DataManager.get_original_volume_filepath_v2(stack_spec=stack_spec, structure=None))

    save_data(volume_origin_wrt_wholebrain_outVolResol, 
              fp=DataManager.get_original_volume_origin_filepath_v3(stack_spec=stack_spec, structure=None))
    
    ##########################################
    
    display_volume_sections(volume_outVolResol, cmap=plt.cm.gray, every=20, title_fontsize=20)
    
    ###########################################

# Crop, generate prep2 raw from prep5

In [ ]:
from multiprocess import Pool

In [ ]:
def f(stack, img_name, version, resol, x,y,w,h):

    input_fp = DataManager.get_image_filepath_v2(stack=stack, prep_id=5, resol=resol, version=version, fn=img_name)
    output_fp = DataManager.get_image_filepath_v2(stack=stack, fn=img_name, prep_id=2, version=version, resol=resol)

    img = imread(input_fp)
    save_data(img[y:y+h, x:x+w], output_fp)

In [ ]:
# for stack in ['CHATM2', 'CHATM3']:
for stack in ['MD589']:

    alignedBrainstemCrop_xmin, alignedBrainstemCrop_xmax, \
    alignedBrainstemCrop_ymin, alignedBrainstemCrop_ymax = \
    DataManager.load_cropbox_v2(stack=stack, prep_id='alignedBrainstemCrop', only_2d=True)

    alignedWithMargin_xmin, alignedWithMargin_xmax,\
    alignedWithMargin_ymin, alignedWithMargin_ymax = DataManager.load_cropbox_v2(stack=stack, anchor_fn=None, 
                                                            prep_id='alignedWithMargin',
                                                           return_dict=False, only_2d=True)

    alignedBrainstemCrop_xmin_wrt_alignedWithMargin = alignedBrainstemCrop_xmin - alignedWithMargin_xmin
    alignedBrainstemCrop_xmax_wrt_alignedWithMargin = alignedBrainstemCrop_xmax - alignedWithMargin_xmin
    alignedBrainstemCrop_ymin_wrt_alignedWithMargin = alignedBrainstemCrop_ymin - alignedWithMargin_ymin
    alignedBrainstemCrop_ymax_wrt_alignedWithMargin = alignedBrainstemCrop_ymax - alignedWithMargin_ymin

    print alignedBrainstemCrop_xmin_wrt_alignedWithMargin,\
    alignedBrainstemCrop_xmax_wrt_alignedWithMargin,\
    alignedBrainstemCrop_ymin_wrt_alignedWithMargin,\
    alignedBrainstemCrop_ymax_wrt_alignedWithMargin

    x_tb = alignedBrainstemCrop_xmin_wrt_alignedWithMargin
    y_tb = alignedBrainstemCrop_ymin_wrt_alignedWithMargin
    w_tb = alignedBrainstemCrop_xmax_wrt_alignedWithMargin - alignedBrainstemCrop_xmin_wrt_alignedWithMargin + 1
    h_tb = alignedBrainstemCrop_ymax_wrt_alignedWithMargin - alignedBrainstemCrop_ymin_wrt_alignedWithMargin + 1

#     for version in ['NtbNormalizedAdaptiveInvertedGamma', 'CHAT']:
#     for version in ['CHAT']:
    for version in ['NtbNormalizedAdaptiveInvertedGamma']:
#     for version in ['mask']:
#         for resol in ['thumbnail']:
        for resol in ['raw']:
            
            if resol == 'raw':
                x = x_tb * 32
                y = y_tb * 32
                w = w_tb * 32
                h = h_tb * 32
            elif resol == 'thumbnail':
                x = x_tb
                y = y_tb
                w = w_tb
                h = h_tb
            else:
                raise

#             input_dir = DataManager.get_image_dir_v2(stack=stack, prep_id=5, version=version, resol='raw')
            out_dir = DataManager.get_image_dir_v2(stack=stack, prep_id=2, resol=resol, version=version)
            print 'out_dir:', out_dir
#             script = os.path.join(REPO_DIR, 'preprocess', 'warp_crop_IM_v3.py')

    #         ! rm -rf {out_dir}
            create_if_not_exists(out_dir)

            t = time.time()

            pool = Pool(8)
            _ = pool.map(lambda img_name: f(stack=stack, img_name=img_name, version=version, resol=resol, 
                                            x=x, y=y, w=w, h=h), 
                         metadata_cache['valid_filenames'][stack])
            pool.close()
            pool.join()

#             for img_name in metadata_cache['valid_filenames'][stack]:
#                 f(stack=stack, img_name=img_name, version=version, resol=resol, 
#                                             x=x, y=y, w=w, h=h)

        #     run_distributed('convert \"%%(input_fp)s\" -crop %(w)dx%(h)d+%(x)d+%(y)d  \"%%(output_fp)s\"' % \
        #                     {'w':w_raw, 'h':h_raw, 'x':x_raw, 'y':y_raw},
        #                     kwargs_list=[{'input_fp': DataManager.get_image_filepath_v2(stack=stack, prep_id=5, resol='raw', version=version, fn=img_name),
        #                                   'output_fp': DataManager.get_image_filepath_v2(stack=stack, fn=img_name, prep_id=2, version=version, resol='raw')}
        #                                  for img_name in metadata_cache['valid_filenames'][stack]],
        # #                                  for img_name in ['CHATM3_slide35_2018_02_17-S1']],
        #                     argument_type='single',
        #                    jobs_per_node=1,
        #                    local_only=True)

            # wait_qsub_complete()

            print 'done in', time.time() - t, 'seconds' # 1500s

# Transform original masks to prep1
thumbnail_mask -> prep1_thumbnail_mask

In [ ]:
# Legacy data folder names do not match current scheme.
# First rename MD589_masks as MD589_thumbnail_mask. 

In [27]:
download_from_s3('/data/CSHL_data_processed/MD589/MD589_masks/', 
                 is_dir=True)

rm -rf "/data/CSHL_data_processed/MD589/MD589_masks" && mkdir -p "/data/CSHL_data_processed/MD589"
return code: 0
aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_masks" "/data/CSHL_data_processed/MD589/MD589_masks"
return code: 0


In [30]:
! mv /data/CSHL_data_processed/MD589/MD589_masks /data/CSHL_data_processed/MD589/MD589_thumbnail_mask

In [31]:
for img_name in valid_filenames:
    os.system("mv /data/CSHL_data_processed/MD589/MD589_thumbnail_mask/%s_mask.png /data/CSHL_data_processed/MD589/MD589_thumbnail_mask/%s_thumbnail_mask.png" % (img_name, img_name))

In [42]:
upload_to_s3(DataManager.get_image_dir_v2(stack=stack, prep_id=None, version='mask', resol='thumbnail'), 
             is_dir=True, local_root=DATA_ROOTDIR)

aws s3 cp --recursive "/data/CSHL_data_processed/MD589/MD589_thumbnail_mask" "s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_thumbnail_mask"
return code: 0


In [ ]:
# Do transform thumbnail_mask -> prep1_thumbnail_mask

In [55]:
download_from_s3(DataManager.get_image_dir_v2(stack=stack, prep_id=None, version='mask', resol='thumbnail'), 
             is_dir=True, local_root=DATA_ROOTDIR, redownload=True)

rm -rf "/data/CSHL_data_processed/MD589/MD589_thumbnail_mask" && mkdir -p "/data/CSHL_data_processed/MD589"
return code: 0
aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_thumbnail_mask" "/data/CSHL_data_processed/MD589/MD589_thumbnail_mask"
return code: 0


In [56]:
stack = 'MD589'

In [57]:
anchor_fn = DataManager.load_anchor_filename(stack=stack)

In [58]:
transforms_fp = os.path.join(DATA_DIR, stack, '%(stack)s_transformsTo_%(anchor_fn)s.pkl' % \
                                                dict(stack=stack, anchor_fn=anchor_fn))
download_from_s3(transforms_fp, redownload=True)

rm -rf "/data/CSHL_data_processed/MD589/MD589_transformsTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091.pkl" && mkdir -p "/data/CSHL_data_processed/MD589"
return code: 0
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_transformsTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091.pkl" "/data/CSHL_data_processed/MD589/MD589_transformsTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091.pkl"
return code: 0


In [59]:
pad_bg_color = 'black'

In [60]:
prep_id = 1

In [61]:
version = 'mask'

In [62]:
input_dir = DataManager.get_image_dir_v2(stack=stack, prep_id=None, version=version, resol='thumbnail')
out_dir = DataManager.get_image_dir_v2(stack=stack, prep_id=prep_id, resol='thumbnail', version=version)
print 'out_dir:', out_dir
script = os.path.join(REPO_DIR, 'preprocess', 'warp_crop_IM_v3.py')

out_dir: /data/CSHL_data_processed/MD589/MD589_prep1_thumbnail_mask


In [63]:
! rm -rf {out_dir}

In [64]:
_, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack) 
valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

In [65]:
t = time.time()
print 'Warping...'

transforms_to_anchor = DataManager.load_transforms(stack=stack, downsample_factor=32, use_inverse=False, anchor_fn=anchor_fn)

if pad_bg_color == 'auto': # useful for alternatively stained stacks where bg varies depending on stain on each section
    run_distributed('%(script)s %(stack)s \"%%(input_fp)s\" \"%%(output_fp)s\" %%(transform)s thumbnail 0 0 2000 1500 %%(pad_bg_color)s' % \
                    {'script': script,
                    'stack': stack,
                    },
                    kwargs_list=[{'transform': ','.join(map(str, transforms_to_anchor[fn].flatten())),
                                'input_fp': DataManager.get_image_filepath_v2(stack=stack, fn=fn, prep_id=None, version=version, resol='thumbnail'),
                                  'output_fp': DataManager.get_image_filepath_v2(stack=stack, fn=fn, prep_id=prep_id, version=version, resol='thumbnail'),
                                'pad_bg_color': 'black' if fn.split('-')[1][0] == 'F' else 'white'}
                                for fn in valid_filenames],
                    argument_type='single',
                   jobs_per_node=8,
                   local_only=True)
else:
    run_distributed('%(script)s %(stack)s \"%%(input_fp)s\" \"%%(output_fp)s\" %%(transform)s thumbnail 0 0 2000 1500 %(pad_bg_color)s' % \
                    {'script': script,
                    'stack': stack,
                    'pad_bg_color': pad_bg_color},
                    kwargs_list=[{'transform': ','.join(map(str, transforms_to_anchor[fn].flatten())),
                                'input_fp': DataManager.get_image_filepath_v2(stack=stack, fn=fn, prep_id=None, version=version, resol='thumbnail'),
                                  'output_fp': DataManager.get_image_filepath_v2(stack=stack, fn=fn, prep_id=prep_id, version=version, resol='thumbnail'),
                                 }
                                for fn in valid_filenames],
                    argument_type='single',
                   jobs_per_node=8,
                   local_only=True)

# wait_qsub_complete()
    
print 'done in', time.time() - t, 'seconds' # 300 seconds.

Warping...


rm -f ~/stderr_*; rm -f ~/stdout_*
return code: 0
Run locally.


done in 118.715882063 seconds


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


In [66]:
upload_to_s3(out_dir, is_dir=True, local_root=DATA_ROOTDIR)

aws s3 cp --recursive "/data/CSHL_data_processed/MD589/MD589_prep1_thumbnail_mask" "s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_prep1_thumbnail_mask"
return code: 0
